#  The Static Shared Library with GCC

**C/C++ Under Linux**

* %cd Change the current working directory to `./demo`

In [1]:
%cd demo

J:\SEU\SEE\PySEE\home\notebook\demo


In [49]:
%pwd

'J:\\SEU\\SEE\\PySEE\\home\\notebook\\demo'

##  The Library 

Library files are a collection of **precompiled functions** that are written according to the principle of reusability. 

They are usually made up of a set of interrelated functions used to perform a common task

### Static  Library 

Static libraries are **directly linked into** the program at compile time. 

A program using a static library takes copies of the code that it uses from the static library and makes it part of the program. 



### Shared  Library 

For a shared library, the compiler/linker checks that the names you link with exist in the library when the application is built, **but doesn't move their code into the application**. At run time, the shared library must be available.

Before the executable starts running, **the operating system loads the machine code needed for the external functions** - a process known as **dynamic linking.** 

### The advantages and disadvantages in each method

**The advantages of using library**

1. Modular development

2. Reusability

3. Maintainability

**Shared libraries**

* reduce the amount of code that is duplicated in each program that makes use of the library, keeping **binaries small**. 

* allows you to replace the shared object with one that is functionally equivalent, but may have added performance benefits **without needing to recompile** the program that makes use of it. 

Shared libraries will, however have a small additional cost for the execution of the functions as well as a run-time loading cost as all the symbols in the library need to be connected to the things they use

**Static libraries**

* increase the overall size of the binary, but it means that you don't need to carry along a copy of the library that is being used. As the code is connected at compile time there are not any additional run-time loading costs. The code is simply there.

![library](./img/library.png) 




**GCC assumes that all libraries**
   
* `start` with **lib**

* Static libraries: `end` with `.a`

* Shared libraries: `end`  with `.dll`(windows) or `.so`(Linux)，



## The source code of library 
The static and shared library we will build consist of a single source file: 

* `statistics.c/h`


In [2]:
%%file ./include/statistics.h
#ifndef STATISTICS_H
#define STATISTICS_H

double  mean(double data[], int size);

#endif


Overwriting ./include/statistics.h


In [3]:
%%file ./src/statistics.c

#include "statistics.h"

double  mean(double data[], int size)
{
  /* Compute the arithmetic mean of a dataset using the recurrence relation 
     mean_(n) = mean(n-1) + (data[n] - mean(n-1))/(n+1)   */

  double mean = 0;
  for(int  i = 0; i < size; i++)
  {
      mean += (data[i] - mean) / (i + 1);
   }
  return mean;
}

Overwriting ./src/statistics.c


## 1 The Static Library

### Step 1: Compiling Code

In [31]:
!gcc -c -O3 -Wall -o  ./obj/statistics.o  ./src/statistics.c -I./include/

* `-c`: compile into object file with default name : *.o.

      By default, the object file has the same name as the source file with extension of ".o" 
  
  
* `-O3`: Optimize yet more.

      turns on all optimizations specified by -O2 and also turns on the -finline-functions, -fweb, -frename-registers and -funswitch-loops optionsturns  on  all  optimizations   
     
  
* `-Wall`: prints "all"  compiler's warning message. 

      This option should always be used, in order to generate better code.


### Step 2: Creating a static library from the object files

The static file is created with the archiver (`ar`).

 `./lib/libstat.a `

In [32]:
!ar -rv ./lib/libstat.a ./obj/statistics.o 

r - ./obj/statistics.o


* `r`  - replace existing or insert new file(s) into the archive(替换库中存在de模块/插入新模块）

* `v`  - be verbose(该选项用来显示执行操作选项的附加信息)

In [16]:
!dir .\lib\libstat.*

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEE\PySEE\home\notebook\demo\lib 的目录

2021/01/04  15:54               874 libstat.a
               1 个文件            874 字节
               0 个目录 90,554,208,256 可用字节


In [7]:
!del .\obj\statistics.o 

### Step 3 Using a static library 

In [8]:
%%file ./src/statapp.c

#include <stdio.h> 
#include "statistics.h"

int main() {
     double a[] = {8, 4, 5, 3, 2};
     int length = sizeof(a)/sizeof(double);  
     printf("mean is %f\n", mean(a,length)); 
     return 0;
}


Overwriting ./src/statapp.c


In [17]:
!gcc -c -o ./obj/statapp.o ./src/statapp.c -I./include/

In [18]:
!gcc -o  ./bin/statappstatic ./obj/statapp.o -L./lib -lstat

In [19]:
!dir .\bin\statappstatic.*

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEE\PySEE\home\notebook\demo\bin 的目录

2021/01/04  15:54           330,645 statappstatic.exe
               1 个文件        330,645 字节
               0 个目录 90,554,208,256 可用字节


In [20]:
!.\bin\statappstatic.exe

mean is 4.400000


### Header Files and Libraries 

* `Header File`: When compiling the program, the **compiler** needs the **header** files to compile the source codes;

* `libraries`: the **linker** needs the **libraries** to resolve external references from other object files or libraries. 

The `compiler` and `linker` will not find the `headers/libraries` unless you set **the appropriate options**

* **1 Searching for Header Files**

   **`-Idir`:** The include-paths are specified via **-Idir** option (`uppercase` 'I' followed by the directory path. 
   
   
* **2 Searching for libraries Files**

   **`-Ldir`**: The library-path is specified via **-Ldir** option (`uppercase` 'L' followed by the directory path）. 


* **3 Linking the library**

   **`-llibname`**: Link with the library name **without** the `lib` prefix and the `a` extensions.
  
                                                                 
                                                                    


## 2 The shared library




we use the same source file`statistics.c/h` to compile into a shared library 

#### Step 1: Compiling with Position Independent Code

We compile the C file with `Position Independent Code( PIC )` 


* **`-fPIC`** : stands for `Position Independent Code`(位置无关代码)
   
   the generated machine code is `not dependent` on being located at a `specific address` in order to `work`.
   
   Position-independent code can be `executed` at `any memory address`
     

In [21]:
!gcc -c -O3 -Wall -fPIC  -o  ./obj/statistics.o  ./src/statistics.c -I./include/

#### Step 2: Creating a shared library from an object file

* -shared : creating a shared library

In [22]:
!gcc -shared -o  ./bin/libstat.dll ./obj/statistics.o

**Makefile of Building the shared library**

In [23]:
%%file ./makestatso.mk

CC=gcc
CFLAGS=-O3 -Wall -fPIC 

SRCDIR=./src/
OBJDIR=./obj/
BINDIR=./bin/
INCDIR=./include/

# Linux
#LIB=libstat.so
# Windows
LIB=libstat.dll

all:  $(LIB)

$(LIB): statobj
	 $(CC) -shared -o $(BINDIR)$@ $(OBJDIR)statistics.o
	 del .\obj\statistics.o
    
statobj: $(SRCDIR)statistics.c
	 $(CC) -c $(CFLAGS)  -o $(OBJDIR)statistics.o $(SRCDIR)statistics.c -I$(INCDIR)
     


Overwriting ./makestatso.mk


In [24]:
!make -f makestatso.mk

gcc -c -O3 -Wall -fPIC   -o ./obj/statistics.o ./src/statistics.c -I./include/
gcc -shared -o ./bin/libstat.dll ./obj/statistics.o
del .\obj\statistics.o


In [25]:
!dir .\bin\libstat*.*

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEE\PySEE\home\notebook\demo\bin 的目录

2021/01/04  15:50               874 libstat-static.a
2021/01/04  15:54           309,891 libstat.dll
               2 个文件        310,765 字节
               0 个目录 90,554,212,352 可用字节


#### Step3: Using the shared library

**Linking the library**

   **`-llibname`**: Link with the library name **without** the `lib` prefix and the `.dll/.so` extensions.
  
   Windows
   ```bash
      -L./bin/ -lstat
  ```
>**Linux**
> ```bash
>   -L./bin/ -lstat -Wl,-rpath=./bin/  
> ```
>* **`-Wl,option`**
>
>    Pass option as an option to the **linker**. 
>    
>    If option contains `commas`, it is split into multiple options at the commas. 
>    
>    You can use this syntax to pass an argument to the option.
>      
>   
>* **`-rpath=dir`** 
>
>   **Add a directory to the runtime library search path**. This is used when linking an >executable with shared objects. All -rpath arguments are concatenated and passed to the >runtime linker, which uses them to locate shared objects at runtime. 
   


We use the **same** code ` ./src/statApp.c` in static example to demonstrates calling the shared library 



**Makefile**

In [26]:
%%file ./makestatsoapp.mk

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR=./include/

all: statappso

statappso: statobj  
	gcc -o $(BINDIR)$@ $(OBJDIR)statapp.o -L$(BINDIR) -lstat
#	gcc -o $(BINDIR)$@ $(OBJDIR)statApp.o -L$(BINDIR) -lstat -Wl,-rpath=./bin/  

statobj: $(SRCDIR)statapp.c 
	gcc -c -o $(OBJDIR)statapp.o $(SRCDIR)statapp.c -I$(INCDIR)

Overwriting ./makestatsoapp.mk


In [27]:
!make -f makestatsoapp.mk

gcc -c -o ./obj/statapp.o ./src/statapp.c -I./include/
gcc -o ./bin/statappso ./obj/statapp.o -L./bin/ -lstat


In [28]:
!dir .\bin\statapp*.*

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEE\PySEE\home\notebook\demo\bin 的目录

2021/01/04  15:55           330,674 statappso.exe
2021/01/04  15:54           330,645 statappstatic.exe
               2 个文件        661,319 字节
               0 个目录 90,554,212,352 可用字节


In [29]:
!.\bin\statappSo

mean is 4.400000


## 3 Building  a Shared Library with Multi-source

### 3.1  The multi-source files
The shared library we will build consist of the multi-source files

* statistics.c/h
* funs.c/h

**funs.c/h N!**

In [69]:
%%file ./include/funs.h

#ifndef FUNS_H
#define FUNS_H

int factorial(int n);

#endif

Overwriting ./include/funs.h


In [70]:
%%file ./src/funs.c

#include "funs.h"

// The factorial of a positive integer n, denoted by n!, 
//    is the product of all positive integers less than or equal to n. 
//  For example,5!=5*4*3*2*1=120
//  The value of 0! is 1 
int factorial(int n)
{
    if (n == 0 ) {
        return 1;
    }
    else 
    {
        return n * factorial(n - 1);
    }
}

Overwriting ./src/funs.c


### 3.2 Building  into One Shared Library

**Makefile**

```make
SRCS=$(SRCDIR)statistics.c \
    $(SRCDIR)funs.c 
```    

In [71]:
%%file ./makefile-libfuns.mk

CC=gcc
CFLAGS=-O3 -Wall -fPIC

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR= ./include/

# Linux
# LIB=libfuns.so 
LIB=libfuns.dll

SRCS=$(SRCDIR)statistics.c \
    $(SRCDIR)funs.c 

# non-path filename
filename=$(notdir $(SRCS))

# the obj target of a source code using the pattern rule
OBJS=$(patsubst %.c,$(OBJDIR)%.o,$(filename))

all:$(LIB)
    
$(LIB): $(OBJS)  
	$(CC) -shared -o $(BINDIR)$@ $(OBJS) 

# the pattern rule: one step rule for multiple source files
$(OBJS):$(SRCS)
	$(CC) $(CFLAGS) -o $@ -c $(SRCDIR)$(patsubst  %.o,%.c,$(notdir $@))  -I$(INCDIR)

Overwriting ./makefile-libfuns.mk


In [72]:
!make -f makefile-libfuns.mk

gcc -O3 -Wall -fPIC -o obj/statistics.o -c ./src/statistics.c  -I./include/
gcc -O3 -Wall -fPIC -o obj/funs.o -c ./src/funs.c  -I./include/
gcc -shared -o ./bin/libfuns.dll ./obj/statistics.o ./obj/funs.o 


In [73]:
!dir .\bin\libf*.dll

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEE\PySEE\home\notebook\demo\bin 的目录

2021/01/03  16:47           310,189 libfuns.dll
               1 个文件        310,189 字节
               0 个目录 90,840,621,056 可用字节


### 3.3 Building a client executable 

The source code `"funsApp.c"` demonstrates calling the shared library's functions:


In [74]:
%%file ./src/funsApp.c

#include <stdio.h> 
#include "statistics.h"
#include "funs.h"

int main() {
    
     double a[] = {8, 4, 5, 3, 2};
     int len= sizeof(a)/sizeof(*a);
     printf("mena is %f\n", mean(a,  len)); 
    
     int n =5;
     printf("the factorial of  %d is %d\n",n,factorial(n));  // 5!=120
     return 0;
}

Overwriting ./src/funsApp.c


#### Windows 

In [75]:
!gcc -c -o ./obj/funsApp.o ./src/funsApp.c -I./include/
!gcc -o  ./bin/funsApp ./obj/funsApp.o  -L./bin/ -lfuns

In [76]:
!.\bin\funsApp

mena is 4.400000
the factorial of  5 is 120


#### Linux

In [ ]:
!gcc -c -o ./obj/funsApp.o ./src/funsApp.c -I./include/
!gcc -o  ./bin/funsApp ./obj/funsApp.o  -L./bin/ -lfuns -Wl,-rpath=./bin/ 

In [ ]:
!./bin/funsApp

## 4 The C/C++ Caller of CoolProp Library in Windows

**Change working dir to `./notebook/coolpropdemo`**


In [1]:
%pwd

'J:\\SEU\\SEE\\PySEE\\home\\notebook'

In [2]:
%cd coolpropdemo

J:\SEU\SEE\PySEE\home\notebook\coolpropdemo


### 4.1 The Static CoolProp Library

* The Static CoolProp library(64) on Windows: `./lib/libCoolProp.a` （the size is is too large,so it is not provided in the PySEE/home) 

In [3]:
%%file ./mainstatic.c

/* 
The example use the static library of CoolProp in c

The library: ./lib/libCoolProp.a

g++ -o ./bin/mainstatic.exe   main.c -I./include -L./lib -lCoolProp
       
./bin/mainstatic

*/

#include "CoolPropLib.h"
#include <stdio.h>

int main() {
     double value=PropsSI("P","T",273.15+0,"Q",0,"R134a");
     printf("%f\n", value); 
     return 0;
}


Overwriting ./mainstatic.c


In [4]:
!g++ -o ./bin/mainstatic.exe   mainstatic.c -I./include -L./lib -lCoolProp

In [9]:
!.\bin\mainstatic.exe

292803.182339


### 4.2 The C/C++ Caller of CoolProp Shared Library in Windows

* [Official Doc: CoolProp Shared Library](http://www.coolprop.org/coolprop/wrappers/SharedLibrary/index.html#)

The example use a MinGW64 64-bit dll: **libCoolProp.dll** of CoolProp

**The Windows Shared Library**

1. The official Windows Shared Library: `CoolProp_x64.dll`

2. The Shared Library with MinGW64: `libCoolProp.dll`



#### C Caller

In [ ]:
# %load ./main.c

/* 
The example use the shared library of CoolProp in c

The library: libCoolProp.dll or CoolProp_x64.dll

g++ -o ./bin/main.exe  -DCOOLPROP_LIB main.c -I./include -L./bin -lCoolProp
       
./bin/main

*/

#include "CoolPropLib.h"
#include <stdio.h>

int main() {
     double value=PropsSI("P","T",273.15+0,"Q",0,"R134a");
     printf("%f\n", value); 
     return 0;
}


Compiling

In [39]:
!g++  -o ./bin/main  -DCOOLPROP_LIB main.c -I./include -L./bin -lCoolProp

Run

In [40]:
!.\bin\main

292803.182339


#### C++ Caller

In [ ]:
# %load ./demo.cpp
/* 
The example use the shared library of CoolProp in c

The library: libCoolProp.dll or CoolProp_x64.dll

g++  -o ./bin/demo.exe  -DCOOLPROP_LIB demo.cpp -I./include -L./bin -lCoolProp

./bin/demo

*/

#include "CoolPropLib.h"
#include <iostream>
#include <iomanip>

int main()
{
    double value=PropsSI("P","T",273.15+0,"Q",0,"R134a");
    std::cout << std::setiosflags(std::ios::fixed)  << value << std::endl;
    return 1;
}

Compiling

In [6]:
!g++  -o ./bin/demo.exe  -DCOOLPROP_LIB demo.cpp -I./include -L./bin -lCoolProp

Run

In [7]:
!.\bin\demo

292803.182339


### 4.3 The Size of Callers 

In [8]:
!dir .\bin\*.exe

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEE\PySEE\home\notebook\coolpropdemo\bin 的目录

2021/01/04  02:26         3,134,599 demo.exe
2021/01/04  02:21           330,685 main.exe
2021/01/04  02:25        28,187,475 mainstatic.exe
               3 个文件     31,652,759 字节
               0 个目录 90,392,883,200 可用字节
